# Python for Creating CSV and Preliminary Reporting

In [1]:
import csv
import contextily as cx
import geopandas as gpd
import json
import matplotlib.pyplot as plt
import pandas as pd
from shapely.geometry import Point
import urllib


#import geopandas as gpd

%matplotlib inline
plt.style.use('ggplot')

from datetime import date
print("Last run: ", date.today())

#gv.extension('bokeh', 'matplotlib')

Last run:  2022-09-17


## Load Data and Write to CSV


In [2]:
with open("roman-amphitheaters.geojson") as f:
    j = json.load(f)

In [3]:
# If there is one, I'd welcome a more pythonic approach. One that 
# accomodates the variable data model supported by JSON.

d = []
for feature in j['features']:
    
    # Check for optional properties

    if 'latintoponym' in feature['properties'].keys():
        latintoponym = feature['properties']['latintoponym']
    else:
        latintoponym = ''    

    if 'welchid' in feature['properties'].keys():
        welchid = feature['properties']['welchid']
    else:
        welchid = ''

    if 'golvinid' in feature['properties'].keys():
        golvinid = feature['properties']['golvinid']
    else:
        golvinid = ''

    if 'buildingtype' in feature['properties'].keys():
        buildingtype = feature['properties']['buildingtype']
    else:
        buildingtype = ''
        
    try:
        zotero = feature['properties']['bibliography'][0]['zoteroitem']
    except:
        zotero = ''
 
    if 'chronogroup' in feature['properties'].keys():
        chronogroup = feature['properties']['chronogroup']
    else:
        chronogroup = ''

    secondcentury = True
    if 'exclude' in feature['properties'].keys():
        secondcentury = False

    if 'capacity' in feature['properties'].keys():
        capacity = feature['properties']['capacity']['quantity']
    else:
        capacity = ''

    if 'province' in feature['properties'].keys():
        romanregion = feature['properties']['province']
    elif 'region' in feature['properties'].keys():
        romanregion = feature['properties']['region']
    else:
        romanregion = ''
        
    arenamajor = ''
    arenaminor = ''
    extmajor = ''
    extminor = ''
    exteriorheight = ''
    dimensionsunknown = False
    if 'dimensions' in feature['properties'].keys():
        dimensions = feature['properties']['dimensions']
        
        if 'arenamajor' in dimensions:
            arenamajor = dimensions['arenamajor']

        if 'arenaminor' in dimensions:
            arenaminor = dimensions['arenaminor']
            
        if 'exteriormajor' in dimensions:
            extmajor = dimensions['exteriormajor']

        if 'exteriorminor' in dimensions:
            extminor = dimensions['exteriorminor']
            
        if 'exteriorheight' in dimensions:
            exteriorheight = dimensions['exteriorheight']
            
        if 'unknown' in dimensions:
            dimensionsunknown = True
            
    d.append((feature['id'],
              feature['properties']['title'],
              feature['properties']['label'],
              latintoponym,
              feature['properties']['pleiades'],
              welchid,
              golvinid,
              buildingtype,
              chronogroup,
              secondcentury,
              capacity,
              feature['properties']['moderncountry'],
              romanregion,
              zotero,
              dimensionsunknown,
              arenamajor,
              arenaminor,
              extmajor,
              extminor,
              exteriorheight,
              feature['geometry']['coordinates'][0],
              feature['geometry']['coordinates'][1],
              feature['geometry']['coordinates'][2]))

ramphs_df = pd.DataFrame(d, columns=(
 'id',    # short id
 'title', # longer title
 'label', # short label
 'latintoponym', # latin toponym
 'pleiades', # pleiades https uri
 'welchid',  # id in Welch
 'golvinid', # id in Golvin
 'buildingtype',  # usually 'amphitheater'
 'chronogroup',   # label for the chronological group
 'secondcentury', # is this an amphitheater that was in use in 2nd century
 'capacity',    # capacity as integer
 'modcountry',  # modern country
 'romanregion', # province or augustan region of italy
 'zotero',      # zotero item id
 'dimensionsunknown', # if true, that's a direct statement that no dimensions are available
 'arenamajor', # long axis of arena in meters
 'arenaminor', # short axis of arena in meters
 'extmajor',   # long axis of exterior
 'extminor', # short axis of exterior
 'exteriorheight',   # height of exterior wall if known
 'longitude', # longitude
 'latitude', # latitude
 'elevation'  # elevation in meters.
 )) 

ramphs_df[['capacity','elevation','arenamajor','arenaminor',
        'extmajor','extminor','exteriorheight']] = ramphs_df[['capacity','elevation','arenamajor',
        'arenaminor','extmajor','extminor','exteriorheight']].apply(pd.to_numeric)

In [4]:
ramphs_df.to_csv("roman-amphitheaters.csv", index = False, quoting = csv.QUOTE_NONNUMERIC)

In [5]:
#ramphs_df[['id','title','chronogroup','latintoponym','romanregion','modcountry','capacity',
#           'extmajor','extminor','arenamajor','arenaminor','latitude','longitude']].to_csv('tmp.csv', index = False, quoting = csv.QUOTE_NONNUMERIC)

## Basic Mapping

In [6]:
ramphs_gdf = gpd.read_file("roman-amphitheaters.geojson")

ramphs_gdf_epsg3857 = ramphs_gdf.to_crs(3857)

ax = ramphs_gdf_epsg3857.plot(figsize=(20, 20), color = 'red')
cx.add_basemap(ax, source=cx.providers.Esri.WorldTerrain)

ax.set_axis_off()
plt.savefig("all-roman-amphitheaters-map.png",pad_inches=0, bbox_inches='tight')

In [7]:
# with roman territory using geopandas


url = "http://sebastianheath.com/roman-maps/roman_empire_ad_200_extent.geojson"
borders_gdf = gpd.read_file(url)

base = borders_gdf.plot(color = 'black', figsize = (40,24))
ramphs_gdf.plot(ax = base, color = 'red', markersize = 100)

plt.grid(False)
plt.axis('off')

plt.savefig('all-roman-amphitheaters-map-dark.png',pad_inches=0, bbox_inches='tight', transparent = True)
plt.close()

## Basic Reporting

In [8]:
ramphs_df.head(2)

id                         title  label  latintoponym  \
0  duraEuroposAmphitheater  Amphitheater at Dura Europos   Dura  Dura Europus   
1        arlesAmphitheater         Amphitheater at Arles  Arles       Arelate   

                                  pleiades welchid golvinid  buildingtype  \
0  https://pleiades.stoa.org/places/893989              129  amphitheater   
1  https://pleiades.stoa.org/places/148217              154  amphitheater   

  chronogroup  secondcentury  ...    zotero dimensionsunknown arenamajor  \
0     severan          False  ...  5ADZC245             False       31.0   
1     flavian           True  ...  LD7NNHSG             False       47.0   

  arenaminor  extmajor  extminor  exteriorheight  longitude   latitude  \
0       25.0      50.0      44.0             NaN  40.728926  34.749855   
1       32.0     136.0     107.0             NaN   4.631111  43.677778   

   elevation  
0        223  
1         21  

[2 rows x 23 columns]

In [9]:
ramphs_df.describe()

capacity  arenamajor  arenaminor    extmajor    extminor  \
count    130.000000  171.000000  167.000000  212.000000  191.000000   
mean   12000.753846   57.018129   38.202994   95.993538   77.258168   
std     8991.382469   14.502869    8.722000   30.538218   25.401345   
min     1000.000000   25.000000   19.000000   25.000000   31.000000   
25%     5037.500000   47.000000   32.350000   74.375000   58.900000   
50%     9700.000000   58.000000   39.000000   94.000000   75.000000   
75%    16000.000000   66.000000   42.850000  115.000000   95.250000   
max    50000.000000  104.000000   74.000000  189.000000  156.000000   

       exteriorheight   longitude    latitude    elevation  
count        3.000000  268.000000  268.000000   268.000000  
mean        41.483333   10.567282   42.245199   196.794776  
std          9.859048    9.063517    4.941042   211.555393  
min         32.450000   -8.493330   31.608189  -121.000000  
25%         36.225000    5.325859   38.480299    34.750000  
50%         40.000000   10.890095   42.089044   121.000000  
75%         46.000000   14.184435   45.596976   286.250000  
max         52.000000   40.728926   55.602600  1170.000000

In [10]:
ramphs_df[ramphs_df.secondcentury].describe()

capacity  arenamajor  arenaminor    extmajor    extminor  \
count    120.000000  161.000000  157.000000  201.000000  182.000000   
mean   11700.816667   57.082919   38.340127   95.851393   77.247308   
std     8852.791643   14.642518    8.813463   29.421104   25.175439   
min     1200.000000   25.000000   19.000000   25.000000   31.000000   
25%     5000.000000   47.000000   32.700000   75.000000   59.100000   
50%     9000.000000   58.000000   39.000000   94.000000   75.000000   
75%    15206.500000   67.000000   43.000000  114.000000   94.375000   
max    50000.000000  104.000000   74.000000  189.000000  156.000000   

       exteriorheight   longitude    latitude    elevation  
count        2.000000  246.000000  246.000000   246.000000  
mean        42.225000   10.046911   42.357248   197.150407  
std         13.823938    8.856168    4.991714   211.790862  
min         32.450000   -8.493330   31.608189     1.000000  
25%         37.337500    4.091795   38.686097    35.750000  
50%         42.225000   10.572890   42.270997   123.500000  
75%         47.112500   14.053709   45.744273   284.500000  
max         52.000000   38.273763   55.602600  1170.000000

In [11]:
# Confirm that CSV is readable
# It would be nice if the "numeric pattern" string survived as strings.
pd.read_csv("roman-amphitheaters.csv", quoting = 2).describe()

welchid    golvinid      capacity  arenamajor  arenaminor  \
count  18.000000   96.000000    130.000000  171.000000  167.000000   
mean    9.777778  108.041667  12000.753846   57.018129   38.202994   
std     5.704029   61.381279   8991.382469   14.502869    8.722000   
min     1.000000   11.000000   1000.000000   25.000000   19.000000   
25%     5.250000   63.500000   5037.500000   47.000000   32.350000   
50%     9.500000  104.500000   9700.000000   58.000000   39.000000   
75%    14.500000  143.750000  16000.000000   66.000000   42.850000   
max    19.000000  298.000000  50000.000000  104.000000   74.000000   

         extmajor    extminor  exteriorheight   longitude    latitude  \
count  212.000000  191.000000        3.000000  268.000000  268.000000   
mean    95.993538   77.258168       41.483333   10.567282   42.245199   
std     30.538218   25.401345        9.859048    9.063517    4.941042   
min     25.000000   31.000000       32.450000   -8.493330   31.608189   
25%     74.375000   58.900000       36.225000    5.325859   38.480299   
50%     94.000000   75.000000       40.000000   10.890095   42.089044   
75%    115.000000   95.250000       46.000000   14.184435   45.596976   
max    189.000000  156.000000       52.000000   40.728926   55.602600   

         elevation  
count   268.000000  
mean    196.794776  
std     211.555393  
min    -121.000000  
25%      34.750000  
50%     121.000000  
75%     286.250000  
max    1170.000000

In [12]:
# which have heights
ramphs_df[ramphs_df.exteriorheight > 0]

id                         title           label  \
4   romeFlavianAmphitheater  Flavian Amphitheater at Rome       Colosseum   
90     thysdrusAmphitheater      Amphitheater at Thysdrus  Thysdrus (lg.)   
97         pulaAmphitheater          Amphitheater at Pula            Pula   

                                      latintoponym  \
4                                                    
90                                                   
97  Colonia Pietas Iulia Pola Pollentia Herculanea   

                                   pleiades welchid golvinid  buildingtype  \
4   https://pleiades.stoa.org/places/423025              152  amphitheater   
90  https://pleiades.stoa.org/places/324835                   amphitheater   
97  https://pleiades.stoa.org/places/197448                   amphitheater   

       chronogroup  secondcentury  ...    zotero dimensionsunknown arenamajor  \
4          flavian           True  ...  VKMRD8AT             False      83.00   
90    post-severan          False  ...                       False      65.00   
97  julio-claudian           True  ...                       False      67.95   

   arenaminor  extmajor  extminor  exteriorheight  longitude   latitude  \
4       48.00    189.00     156.0           52.00  12.492269  41.890169   
90      39.00    148.00     122.0           40.00  10.706939  35.296390   
97      41.65    132.45     105.1           32.45  13.850243  44.873229   

    elevation  
4          22  
90        111  
97         16  

[3 rows x 23 columns]

In [13]:
# which don't have exteriormajor
ramphs_df[pd.isnull(ramphs_df.extmajor)].sort_values(by = 'longitude')\
[['id','modcountry','latintoponym','golvinid','extmajor','arenamajor','latitude','longitude']]

id      modcountry  \
157                        bragaAmphitheater        Portugal   
166                     bobadelaAmphitheater        Portugal   
56                         lixusAmphitheater         Morocco   
121                      caparraAmphitheater           Spain   
159                         leonAmphitheater           Spain   
193                       espejoAmphitheater           Spain   
158                       sisapoAmphitheater           Spain   
266                       toledoAmphitheater           Spain   
87                   cirencesterAmphitheater  United Kingdom   
68                    aldburoughAmphitheater  United Kingdom   
254                         yorkAmphitheater  United Kingdom   
46                        gennesAmphitheater          France   
143                         agenAmphitheater          France   
182                    chassenonAmphitheater          France   
150              caistorStEdmundAmphitheater  United Kingdom   
147                     chartresAmphitheater          France   
37                         parisAmphitheater          France   
202           montbouyGalloRomanAmphitheater          France   
146                        meauxAmphitheater          France   
127                        reimsAmphitheater          France   
205                  mesarafeltaAmphitheater         Algeria   
20                      nijmegenAmphitheater     Netherlands   
62   hydataHypsitanaForumTraianiAmphitheater           Italy   
191                     tebourbaAmphitheater         Tunisia   
207                       carpisAmphitheater         Tunisia   
198                       pupputAmphitheater         Tunisia   
258                 reggioEmiliaAmphitheater           Italy   
262                       trentoAmphitheater           Italy   
203              statiliusTaurusAmphitheater           Italy   
3                           ludusMagnusArena           Italy   
225                   forumNovumAmphitheater           Italy   
83                        assisiAmphitheater           Italy   
233                        fermoAmphitheater           Italy   
251                 suessaAruncaAmphitheater           Italy   
153                      liternoAmphitheater           Italy   
240                        teanoAmphitheater           Italy   
13                 pozzuoliEarlyAmphitheater           Italy   
239                       chietiAmphitheater           Italy   
151              capuaRepublicanAmphitheater           Italy   
139                    beneventoAmphitheater           Italy   
252                   altripaldaAmphitheater           Italy   
253                       compsaAmphitheater           Italy   
172                    gleisdorfAmphitheater         Austria   
238                      sipontoAmphitheater           Italy   
22                            carnuntumLudus         Austria   
236                      egnaziaAmphitheater           Italy   
39                        cyreneAmphitheater           Libya   
118                       sophiaAmphitheater        Bulgaria   
204                      hisaryaAmphitheater        Bulgaria   
261                      nazilliAmphitheater          Turkey   
221                        geyreAmphitheater          Turkey   
256                     istanbulAmphitheater          Turkey   
105                      salamisAmphitheater          Cyprus   
47               scythopolisNysaAmphitheater          Israel   
102                      antiochAmphitheater          Turkey   
175                      palmyraAmphitheater           Syria   

                     latintoponym golvinid  extmajor  arenamajor   latitude  \
157               Bracara Augusta                NaN         NaN  41.546669   
166                     Elbocoris                NaN        50.0  40.361088   
56                                               NaN        34.0  35.199900   
121  Municipium Flavium Caparense                NaN        30.0  40.164159  

In [14]:
ramphs_df[ramphs_df.golvinid == '' ][['id','latintoponym','golvinid','extmajor','arenamajor','latitude','longitude']]

id          latintoponym golvinid  extmajor  \
2              lyonAmphitheater              Lugdunum              105.0   
3              ludusMagnusArena          Ludus Magnus                NaN   
6    eleutheropolisAmphitheater        Eleutheropolis               71.0   
7           pompeiiAmphitheater               Pompeii              135.0   
9          newsteadAmphitheater            Trimontium               70.0   
..                          ...                   ...      ...       ...   
260  torreparedonesAmphitheater  Colonia Virtus Iulia               69.0   
261         nazilliAmphitheater              Mastaura                NaN   
262          trentoAmphitheater             Tridentum                NaN   
266          toledoAmphitheater               Toletum                NaN   
267    kaiseraugustAmphitheater    Castrum Rauracense               50.0   

     arenamajor   latitude  longitude  
2          67.6  45.770556   4.830556  
3           NaN  41.889950  12.494913  
6          47.0  31.608189  34.893845  
7          66.0  40.751213  14.495262  
9          37.0  55.602600  -2.691900  
..          ...        ...        ...  
260         NaN  37.752468  -4.380600  
261         NaN  37.955596  28.338099  
262        62.0  46.069205  11.125334  
266         NaN  39.865349  -4.022888  
267         NaN  47.540822   7.721596  

[172 rows x 7 columns]

In [15]:
ramphs_df[ramphs_df.latintoponym == '' ][['id','latintoponym','golvinid','extmajor','arenamajor','latitude','longitude']]

id latintoponym golvinid  \
4                        romeFlavianAmphitheater                   152   
5                     romeAmphitheatrumCastrense                   189   
19                        porolissumAmphitheater                    39   
28                           italicaAmphitheater                   168   
34                        viminaciumAmphitheater                         
35                       flaviaSolvaAmphitheater                         
38   octodurusForumClaudiiVallensiumAmphitheater                         
39                            cyreneAmphitheater                         
44                      charterhouseAmphitheater                         
47                   scythopolisNysaAmphitheater                         
52                           virunumAmphitheater                         
56                             lixusAmphitheater                         
60                           allifaeAmphitheater                         
62       hydataHypsitanaForumTraianiAmphitheater                         
76                            salonaAmphitheater                   181   
78                             tiburAmphitheater                         
85                         lambaesisAmphitheater                         
86                          mactarisAmphitheater                   119   
88                        vindonissaAmphitheater                    17   
89                     thysdrusSmallAmphitheater                         
90                          thysdrusAmphitheater                         
92                            gortynAmphitheater                         
93                        leptiminusAmphitheater                         
94                             miciaAmphitheater                         
95                          gemellaeAmphitheater                    44   
96                         cherchellAmphitheater                    81   
99                          caesareaAmphitheater                         
101                        anazarbusAmphitheater                         
102                          antiochAmphitheater                         
103                          cyzicusAmphitheater                         
104                         sabrathaAmphitheater                   120   
105                          salamisAmphitheater                         
106                           paphosAmphitheater                         
109                      tomen-y-murAmphitheater                    31   
110                          thaenaeAmphitheater                    71   
116                          bararusAmphitheater                         
117                     castraAlbanaAmphitheater                   190   
119                        ptolemaisAmphitheater                         
158                           sisapoAmphitheater                         
162                         arnsburgAmphitheater                         
163                        ierapetraAmphitheater                         
170                          dambachAmphitheater                         
172                        gleisdorfAmphitheater                         
175                          palmyraAmphitheater                         
188                   genzanoPrivateAmphitheater                         
202               montbouyGalloRomanAmphitheater                         
203                  statiliusTaurusAmphitheater                         
223                                zagaroloLudus                         
224                         tusculumAmphitheater                         
231                      jebelMorabaAmphitheater                         
265               saintGeorgesDuBoisAmphitheater                   103   

     extmajor  arenamajor   latitude  longitude  
4      189.00       83.00  41.890169  12.492269  
5       88.00         NaN  41.887715  12.515144  
19      84.00       64.00  47.177552  23.154052

## Duplicate Checking

In [16]:
dups = ramphs_df[ramphs_df.label.duplicated(keep = False)]\
[['id','pleiades','latintoponym','latitude','longitude']].sort_values('pleiades')

len(dups) == 0

True

In [17]:
dups = ramphs_df[ramphs_df.id.duplicated(keep = False)]\
[['id','pleiades','latintoponym','latitude','longitude']].sort_values('pleiades')

len(dups) == 0

True

In [18]:
dups = ramphs_df[ramphs_df.pleiades.duplicated(keep = False)]\
[['id','pleiades','latintoponym','latitude','longitude']].sort_values('pleiades')

len(dups) == 15

True

In [19]:
dups = ramphs_df[ramphs_df.latintoponym.duplicated(keep = False)]\
[['id','pleiades','latintoponym',
  'latitude','longitude']].sort_values('pleiades')

len(dups.query("latintoponym != ''")) == 11

True

# Chronogrps

In [20]:
c_grps_from_j = j['romanamphitheaterschronogroups']

In [21]:
c_grps_to_csv = [[g['id'],g['startdate'],g['enddate']] for g in c_grps_from_j]

In [22]:
print(f"In c_grps, not in data: {set([i[0] for i in c_grps_to_csv]) - set([i[8] for i in d])}")
print(f"In data, not in c_grps: {set([i[8] for i in d]) - set([i[0] for i in c_grps_to_csv])}")

In c_grps, not in data: set()
In data, not in c_grps: set()


In [23]:
if (set([i[0] for i in c_grps_to_csv]) == set([i[8] for i in d])):
    c_grps_df = pd.DataFrame(c_grps_to_csv, columns = ('id','startdate','enddate'))
    c_grps_df.to_csv("chronogrps.csv", index = False, quoting = csv.QUOTE_NONNUMERIC)

In [24]:
# Provinces (new, incomplete)